# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [27]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [28]:
# your code here
orders = pd.read_csv('Orders.zip')
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [30]:
# your code here
customers_amount_spent = orders.groupby(['CustomerID', 'Country']).agg({'amount_spent' : 'sum'})
customers_amount_spent.head(5)

,,amount_spent
CustomerID,Country,
12346,United Kingdom,77183.60
12347,Iceland,4310.00
12348,Finland,1797.24
12349,Italy,1757.55
12350,Norway,334.40


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [31]:
# your code here
vip_customers = customers_amount_spent[customers_amount_spent['amount_spent'] > customers_amount_spent['amount_spent'].quantile(.95)]
vip_customers.groupby('Country').agg({'amount_spent': 'max'}).sort_values(by=['amount_spent'], ascending=False)


,amount_spent
Country,
Netherlands,280206.02
United Kingdom,259657.30
EIRE,143825.06
Australia,124914.53
Sweden,31906.82
Japan,21429.39
Singapore,21279.29
Germany,19824.05
France,18895.91


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [46]:
# your code here
condition1 = customers_amount_spent['amount_spent'] > customers_amount_spent['amount_spent'].quantile(.75)
condition2 = customers_amount_spent['amount_spent'] < customers_amount_spent['amount_spent'].quantile(.95)
preferred_customers = customers_amount_spent[condition1 & condition2]
preferred_customers

,,amount_spent
CustomerID,Country,
12347,Iceland,4310.00
12348,Finland,1797.24
12349,Italy,1757.55
12352,Norway,2506.04
12356,Portugal,2811.43
...,...,...
18259,United Kingdom,2338.60
18260,United Kingdom,2643.20
18272,United Kingdom,3078.58


In [50]:
combined = pd.concat([vip_customers, preferred_customers]) # 1087
combined.groupby('Country').agg({'amount_spent': 'max'}).sort_values(by=['amount_spent'], ascending=False)

,amount_spent
Country,
Netherlands,280206.02
United Kingdom,259657.30
EIRE,143825.06
Australia,124914.53
Sweden,31906.82
Japan,21429.39
Singapore,21279.29
Germany,19824.05
France,18895.91
